
# Conditional autoregressive transformer

Train a transformer to predict missing pixel from mnist 

### plan

* note to try padded mnist (relative encoding might require black padding???)
* probably don't need positional encoding?
* create transformer model
* masking 
* randomised masking
* relative position encoding (x - current_x, y - current_y, val)
* train to predict when current pixel missing
* train to predict when 10% are missing
* train to predict when 90% are missing
* train to predict when 99% are missing



In [1]:
import tensorflow as tf
import tensorflow.keras as keras

from PIL import Image, ImageDraw
from IPython.display import display
import numpy as np
import itertools

def display_uint8_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display(Image.fromarray(image, "L"))

def display_float32_image(image):
    if len(image.shape) == 3:
        image = image[:, :, 0]
    if tf.is_tensor(image):
        image = image.numpy()
    display_uint8_image(image.astype(np.uint8))





In [2]:
idxs = np.indices([3, 10])
rng = np.random.default_rng()

rng.permutation(idxs[1], axis=1)

array([[8, 6, 4, 3, 1, 5, 9, 0, 2, 7],
       [8, 6, 4, 3, 1, 5, 9, 0, 2, 7],
       [8, 6, 4, 3, 1, 5, 9, 0, 2, 7]])

In [3]:
# reformat MNIST data into (x, y, val) tuples

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

def idxs_to_onehots(idxs):
    onehots = tf.one_hot(idxs, depth=784, dtype=tf.bool, on_value=False, off_value=True)
    return onehots

# takes 2D tensor (batch and index list)
def idxs_to_multihot(idxs):
    onehots = idxs_to_onehots(idxs)
    multihot = tf.reduce_all(onehots, axis=1)
    return multihot

def idxs_to_attention_mask(idxs):
    multihot = idxs_to_multihot(idxs)
    attn_mask = tf.logical_and(multihot[:, :, None], multihot[:, None, :])
    return attn_mask

def mask_to_image_mask(masks):
    image_masks = tf.reshape(masks, [-1, 28, 28])
    return image_masks

# [row, col, value] tuples
def mnist_to_rcv_tuples(dataset):
    # positional encoding is just indices as floats
    x_idx = np.arange(dataset.shape[1]).astype(np.float32)
    y_idx = np.arange(dataset.shape[2]).astype(np.float32)

    # make 28x28 cartesian product and repeat N times
    idxs = np.transpose([np.repeat(y_idx, len(x_idx)),np.tile(x_idx, len(y_idx))])
    
    all_idxs = np.tile(idxs.reshape(-1), dataset.shape[0]).reshape(-1,784,2)
    # pixel data as float between 0 and 255
    all = dataset.astype(np.float32).reshape(-1, 784, 1)
    
    big = np.dstack((all_idxs, all))
    
    return big

def random_offset_tuples(dataset, min_masked_px=0, max_masked_px=784):
    tuples = mnist_to_rcv_tuples(dataset)
    offsets = np.random.randint(0, 28, [dataset.shape[0], 2])
    mask_sizes = np.random.randint(min_masked_px, max_masked_px, [dataset.shape[0]])
    tuples[:, :, 0] -= offsets[:, 0, np.newaxis] # row (y) offset
    tuples[:, :, 1] -= offsets[:, 1, np.newaxis] # col (x) offset
    target_idxs = offsets[:, 0] * 28 + offsets[:, 1]
    
    # permutations 
    masks = np.zeros([dataset.shape[0], 784], dtype=np.bool_)
    for i in range(dataset.shape[0]):
        idxs = np.random.permutation(784)
        n_mask_px = np.random.randint(min_masked_px, max_masked_px)
        # n mask plus always mask the target idx
        mask_idxs = np.concatenate([idxs[:n_mask_px], target_idxs[i:i+1]])
        
        masks[i:i+1] = idxs_to_multihot(mask_idxs[np.newaxis])
    
    target_pxs = dataset[np.arange(dataset.shape[0]), offsets[:, 0], offsets[:, 1]].astype(np.float32)
    return {
        "imgs": tf.cast(dataset, tf.uint8),
        "tuples": tf.cast(tuples, tf.float32),
        "offsets": offsets,
        "target_pxs": target_pxs,
        "target_idxs": target_idxs,
        "masks": tf.cast(masks, tf.bool)
    }

dataset = random_offset_tuples(x_train)

for name, v in dataset.items():
    print(name, "shape", v.shape)

print(dataset["masks"][2])

# should be the same
display_uint8_image(x_train[0])
display_uint8_image(x_train[2])

print(dataset["tuples"][:100, :])

2021-09-03 14:18:32.720371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-03 14:18:32.757347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-03 14:18:32.757691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-03 14:18:32.759540: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

imgs shape (60000, 28, 28)
tuples shape (60000, 784, 3)
offsets shape (60000, 2)
target_pxs shape (60000,)
target_idxs shape (60000,)
masks shape (60000, 784)
tf.Tensor(
[False False  True False  True False  True  True False  True False False
  True False False False  True False  True False False False False False
 False False  True  True  True  True  True  True False  True  True  True
 False False False False False False False False  True False False False
  True False False False False  True  True  True  True  True  True False
  True False False False False False False  True  True False False False
  True False  True False False False  True False  True  True  True  True
 False False False  True  True False False False  True  True  True  True
 False False  True  True  True False False  True  True  True  True  True
 False  True  True False False  True False False  True False False  True
  True False False False False  True False  True False  True False False
  True  True  True False  T

tf.Tensor(
[[[-23. -11.   0.]
  [-23. -10.   0.]
  [-23.  -9.   0.]
  ...
  [  4.  14.   0.]
  [  4.  15.   0.]
  [  4.  16.   0.]]

 [[ -5. -19.   0.]
  [ -5. -18.   0.]
  [ -5. -17.   0.]
  ...
  [ 22.   6.   0.]
  [ 22.   7.   0.]
  [ 22.   8.   0.]]

 [[-16. -12.   0.]
  [-16. -11.   0.]
  [-16. -10.   0.]
  ...
  [ 11.  13.   0.]
  [ 11.  14.   0.]
  [ 11.  15.   0.]]

 ...

 [[-14. -18.   0.]
  [-14. -17.   0.]
  [-14. -16.   0.]
  ...
  [ 13.   7.   0.]
  [ 13.   8.   0.]
  [ 13.   9.   0.]]

 [[-16. -21.   0.]
  [-16. -20.   0.]
  [-16. -19.   0.]
  ...
  [ 11.   4.   0.]
  [ 11.   5.   0.]
  [ 11.   6.   0.]]

 [[ -9. -23.   0.]
  [ -9. -22.   0.]
  [ -9. -21.   0.]
  ...
  [ 18.   2.   0.]
  [ 18.   3.   0.]
  [ 18.   4.   0.]]], shape=(100, 784, 3), dtype=float32)


In [4]:

print(dataset["tuples"].shape, dataset["offsets"].shape, dataset["target_pxs"].shape)
print(dataset["tuples"][0])
print(dataset["offsets"][0])
print(dataset["target_pxs"][0])

# actual training dataset is 784 x,y,v tuples, but with the spatial location changed so the the pixel to predict is in the center
# and the pixel value to predict removed (via masking)

# then, can mask more and more stuff and compare performance

# we can also use the model in an "iterative" mode to generate images, in an arbitrary order.

(60000, 784, 3) (60000, 2) (60000,)
tf.Tensor(
[[-23. -11.   0.]
 [-23. -10.   0.]
 [-23.  -9.   0.]
 ...
 [  4.  14.   0.]
 [  4.  15.   0.]
 [  4.  16.   0.]], shape=(784, 3), dtype=float32)
[23 11]
244.0


In [5]:
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
# example from keras docs https://keras.io/examples/nlp/text_classification_with_transformer/
# uses built-in MultiHeadAttention https://keras.io/api/layers/attention_layers/multi_head_attention/

class TransformerBlock(layers.Layer):
    def __init__(self, n_embed_dim, n_heads, n_dense_dim, dropout_rate, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = layers.MultiHeadAttention(num_heads=n_heads, key_dim=n_embed_dim)
        self.dense_net = keras.Sequential([
            layers.Dense(n_dense_dim, activation='relu'),
            layers.Dense(n_embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
    
    def call(self, inputs, masks, is_training):

        # make mask 2D
        mask = tf.logical_and(masks[:, :, None], masks[:, None, :])
        attn_output = self.att(inputs, inputs, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=is_training)
        out1 = self.layernorm1(inputs + attn_output)
        dense_output = self.dense_net(out1)
        dense_output = self.dropout2(dense_output, training=is_training)
        return self.layernorm2(out1 + dense_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions



In [7]:
tf.constant([True, False]).dtype

tf.bool

In [8]:
def model(batch_size):

    # no batch size to start makes it simpler
    n_embd = 10

    data_input = keras.Input(shape=[784, 3], batch_size=None)
    mask_input = keras.Input(shape=[784], batch_size=None, dtype=tf.bool)
    m = data_input

    # make n_embd-dimensional input embeddings per pixel from [x, y, v]
    m = layers.Dense(20, activation='relu')(m)
    m = layers.Dense(20, activation='relu')(m)
    m = layers.Dense(n_embd, activation=None)(m)
    m = TransformerBlock(n_embed_dim=n_embd, n_heads=7, n_dense_dim=16, dropout_rate=0.1, name="txform1")(m, masks=mask_input, is_training=True)
    m = TransformerBlock(n_embed_dim=n_embd, n_heads=7, n_dense_dim=16, dropout_rate=0.1, name="txform2")(m, masks=mask_input, is_training=True)
    m = layers.Flatten()(m)
    # m = layers.Dense(10, activation='relu')(m)
    m = layers.Dense(1, activation=None)(m)

    model = keras.Model(inputs=[data_input, mask_input], outputs=[m])

    return model


In [9]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Wed_Jul_14_19:41:19_PDT_2021
Cuda compilation tools, release 11.4, V11.4.100
Build cuda_11.4.r11.4/compiler.30188945_0
Fri Sep  3 14:18:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:26:00.0  On |                  N/A |
|  0%   44C    P2    29W / 170W |   4892MiB /  5926MiB |     20%      Default |
|                               |      

In [10]:
batch_size=16

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
lr_metric = get_lr_metric(optimizer)

txformer = model(batch_size)
txformer.compile(optimizer=optimizer, loss='mse', metrics=['MeanAbsoluteError', lr_metric])
load_saved = True
if load_saved:
    txformer.load_weights("./models/txformer-var-mask")

txformer.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784, 3)]     0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 784, 20)      80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 784, 20)      420         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 784, 10)      210         dense_1[0][0]                    
______________________________________________________________________________________________

In [11]:
def fit_one_epoch(dataset):
    txformer.fit(x=[dataset["tuples"], dataset["masks"]], y=dataset["target_pxs"], epochs=1, batch_size=batch_size)

# display:
# - before mask
# - mask
# - after mask
# - prediction
def gen_image(dataset, idx):
    img = np.zeros([28, 28])
    mask = dataset["masks"][idx]
    erow, ecol = dataset["offsets"][idx]

    image_mask = np.array(mask_to_image_mask(mask[np.newaxis, :]), np.uint8)[0]
    masked_image = np.copy(dataset["imgs"][idx])
    
    print("MNIST idx", idx, "which is a", y_train[idx])
    
    display_uint8_image(masked_image) # before mask
    display_uint8_image(image_mask * 255) # mask
    masked_image = masked_image * image_mask
    display_uint8_image(masked_image) # after mask
    
    for row in range(28):
        for col in range(28):
            img_tups = np.copy(dataset["tuples"][idx])
            img_tups[:, 0] += erow
            img_tups[:, 1] += ecol
            img_tups[:, 0] -= row
            img_tups[:, 1] -= col
            mask_idx = row*28+col
            targ_idx_mask = idxs_to_onehots(tf.constant([mask_idx]))
            this_mask = tf.logical_and(mask, targ_idx_mask)
            px = txformer([img_tups[np.newaxis, :, :], this_mask])
            img[row, col] = np.clip(px[0, 0], 0, 255)
            
    display_float32_image(img)

gen_image(dataset, 0)
gen_image(dataset, 1)
gen_image(dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [12]:
fit_one_epoch(dataset)

2021-09-03 14:19:08.700237: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


3750/3750 [==============================] - 250s 66ms/step - loss: 3484.0640 - mean_absolute_error: 36.4051 - lr: 9.9998e-04


In [13]:
gen_image(dataset, 0)
gen_image(dataset, 1)
gen_image(dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [14]:
fit_one_epoch(dataset)

3750/3750 [==============================] - 250s 67ms/step - loss: 1025.8322 - mean_absolute_error: 17.9492 - lr: 9.9998e-04


In [15]:
gen_image(dataset, 0)
gen_image(dataset, 1)
gen_image(dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [16]:
fit_one_epoch(dataset)

3750/3750 [==============================] - 250s 67ms/step - loss: 950.8498 - mean_absolute_error: 16.7752 - lr: 9.9998e-04


In [17]:
gen_image(dataset, 0)
gen_image(dataset, 1)
gen_image(dataset, 2)

MNIST idx 0 which is a 5


MNIST idx 1 which is a 0


MNIST idx 2 which is a 4


In [18]:
txformer.save_weights("./models/txformer-var-mask")